In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Glucocorticoid_Sensitivity/GSE2842'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Additional systems to Prednisolone treated childhood ALL samples"
!Series_summary	"Glucocorticoids (GC) are in most chemotherapy protocols for lymphoid malignancies, particularly childhood acute lymphoblastic leukaemia (ALL) for their ability to induce apoptosis in malignant blast. The underlying mechanism, however, has so far only been investigated in model systems. This study comprises Affymetrix hgu133 plus 2.0 analyses of"
!Series_summary	"Peripheral blood lymphoblasts purified at three time points (0h, 6-8h, 24h after treatment initiation) from 13 children under therapy for ALL . Treated samples were compared to untreated (0h). For comparison, expression profiles were generated from an adult ALL patient, peripheral blood lymphocytes from GC-exposed healthy donors, GC-sensitive and -resistant ALL cell lines and mouse thymocytes treated with GC in vivo and in vitro. This second series comprises the additional samples mentioned above."
!Series_s

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = 0
age_row = 1
gender_row = None  # Gender information is not available
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

def convert_trait(value):
    if 'GC-sensitive' in value:
        return 1
    elif 'GC-resistant' in value:
        return 0
    else:
        return None

def convert_age(value):
    try:
        return float(value.split('aged ')[-1].split(' years')[0])
    except:
        return None

def convert_gender(value):
    value = value.lower().strip()
    if 'male' in value:
        return 1
    elif 'female' in value:
        return 0
    else:
        return None

# Saving cohort information
save_cohort_info('GSE2842', './preprocessed/Glucocorticoid_Sensitivity/cohort_info.json', is_gene_available, trait_row is not None)

# Extract and save selected clinical features
selected_clinical_data = geo_select_clinical_features(clinical_data, 'Glucocorticoid_Sensitivity', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
csv_path = './preprocessed/Glucocorticoid_Sensitivity/trait_data/GSE2842.csv'
selected_clinical_data.to_csv(csv_path)
print(preview_df(selected_clinical_data))


{'GSM60535': [nan, 72.0], 'GSM60536': [nan, 72.0], 'GSM60537': [nan, 72.0], 'GSM60538': [None, None], 'GSM60539': [None, None], 'GSM60540': [None, None], 'GSM60541': [None, None], 'GSM60542': [1, None], 'GSM60543': [1, None], 'GSM60544': [1, None], 'GSM60545': [1, None], 'GSM60546': [1, None], 'GSM60547': [1, None], 'GSM60548': [None, None], 'GSM60549': [None, None], 'GSM60550': [None, None], 'GSM60551': [0, None], 'GSM60552': [0, None], 'GSM60553': [0, None], 'GSM60554': [1, None], 'GSM60555': [1, None], 'GSM60560': [0, None], 'GSM60561': [0, None], 'GSM60562': [0, None], 'GSM60564': [0, None], 'GSM60566': [0, None], 'GSM60576': [0, None], 'GSM60578': [0, None], 'GSM60579': [0, None], 'GSM60581': [1, None], 'GSM60583': [1, None], 'GSM60584': [1, None], 'GSM60586': [1, None], 'GSM60588': [nan, 32.8], 'GSM60589': [nan, 32.8], 'GSM60590': [nan, 32.8], 'GSM60591': [nan, 54.8], 'GSM60592': [nan, 54.8], 'GSM60593': [nan, 54.8], 'GSM60594': [nan, 10.3], 'GSM60595': [nan, 10.3], 'GSM60597': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Identify the key storing probe IDs and the key storing gene symbols
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'
    
    # 2. Get the dataframe storing the mapping between probe IDs and genes
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    
    # 3. Apply the mapping
    gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Glucocorticoid_Sensitivity/gene_data/GSE2842.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Glucocorticoid_Sensitivity')

# 4. Save the cohort information.
save_cohort_info('GSE2842', './preprocessed/Glucocorticoid_Sensitivity/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Glucocorticoid_Sensitivity/GSE2842.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Glucocorticoid_Sensitivity':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Glucocorticoid_Sensitivity' in this dataset is fine.

Quartiles for 'Age':
  25%: nan
  50% (Median): nan
  75%: nan
Min: nan
Max: nan
The distribution of the feature 'Age' in this dataset is fine.

